In [1]:
import pandas as pd

from Classes import Team, Fixture
from definitions import CSV_DIR

In [2]:
liverpool = Team(11)
liverpool.get_home_venue()
liverpool._api_id

40

In [4]:
liverpool.get_prev_fixtures(10)

False
Crystal Palace 1:3 Liverpool. KO: 2022-01-23 14:00:00.
False
Liverpool 3:0 Brentford. KO: 2022-01-16 14:00:00.
False
Chelsea 2:2 Liverpool. KO: 2022-01-02 16:30:00.
False
Leicester 1:0 Liverpool. KO: 2021-12-28 20:00:00.
False
Tottenham 2:2 Liverpool. KO: 2021-12-19 16:30:00.
False
Liverpool 3:1 Newcastle. KO: 2021-12-16 20:00:00.
False
Liverpool 1:0 Aston Villa. KO: 2021-12-11 15:00:00.
False
Wolves 0:1 Liverpool. KO: 2021-12-04 15:00:00.
False
Everton 1:4 Liverpool. KO: 2021-12-01 20:15:00.
False
Liverpool 4:0 Southampton. KO: 2021-11-27 15:00:00.
